In [ ]:
# Parameters
region_name = "Lakshadweep"
models_dir = None
output_dir = None

# Final Visual Showcase for Bathymetry Analysis

This notebook creates comprehensive visualizations for the completed bathymetry analysis pipeline.

Generated visualizations:
1. 📊 Model Performance Comparison
2. 🌈 Synthetic Depth Distribution
3. 🎯 Feature Importance Analysis  
4. 📈 Training Metrics Dashboard
5. 🌐 Interactive Results Summary

## Prerequisites
- Completed model training (03_model_training.ipynb)
- Model files saved in models directory
- Performance metrics available

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path
import joblib

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Add project root to path
project_dir = Path().absolute().parent
if project_dir.name != 'sdb_project':
    project_dir = project_dir / 'sdb_project'
sys.path.append(str(project_dir))

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 150
plt.style.use('default')  # Use default style to avoid seaborn issues

print("✅ Libraries imported successfully")

## Setup and Configuration

Configure paths and load project configuration.

In [ ]:
# Load configuration
config_path = project_dir / 'config' / 'location_config.json'
with open(config_path) as f:
    config = json.load(f)

# Use parameters if provided
if region_name is None:
    region_name = config['region_name']

# Set up paths
region_slug = region_name.lower().replace(' ', '_')
if models_dir is None:
    models_dir = project_dir / 'models'
if output_dir is None:
    output_dir = project_dir / 'outputs' / region_slug

# Create visualization output directory
viz_dir = output_dir / 'visualizations'
viz_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Final showcase for: {region_name}")
print(f"📁 Models directory: {models_dir}")
print(f"📁 Output directory: {output_dir}")
print(f"📁 Visualizations directory: {viz_dir}")

    "print(\"\\n📍 Area of Interest:\")\n",
print(f"   Latitude:  {config['aoi']['min_lat']}° to {config['aoi']['max_lat']}°")
print(f"   Longitude: {config['aoi']['min_lon']}° to {config['aoi']['max_lon']}°")

Project Configuration:
Region: Lakshadweep
Models directory: d:\Project\sdb_project\models
Data directory: d:\Project\sdb_project\data
Outputs directory: d:\Project\sdb_project\outputs

Area of Interest:
  Latitude:  10.75° to 10.95°
  Longitude: 72.35° to 72.65°


In [ ]:
# Load model performance metrics
metrics_path = output_dir / 'model_metrics.json'
comparison_path = output_dir / 'model_performance_comparison.csv'

if not metrics_path.exists():
    raise FileNotFoundError(f"Model metrics not found: {metrics_path}")

# Load metrics
with open(metrics_path) as f:
    model_metrics = json.load(f)

print("✅ Loaded model metrics:")
for model_name, metrics in model_metrics.items():
    print(f"   {model_name}: R² = {metrics['r2_score']:.4f}, RMSE = {metrics['rmse']:.4f}")

# Load comparison CSV if available
if comparison_path.exists():
    metrics_df = pd.read_csv(comparison_path, index_col=0)
        "    print(\"✅ Loaded performance comparison CSV\")\n",
else:
    metrics_df = pd.DataFrame(model_metrics).T
    "    print(\"✅ Created metrics dataframe from JSON\")\n",

# Find best model
best_model_name = metrics_df['r2_score'].idxmax()
best_r2 = metrics_df.loc[best_model_name, 'r2_score']
print(f"\n🏆 Best performing model: {best_model_name} (R² = {best_r2:.4f})")

Loading data and model outputs...


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Project\\sdb_project\\outputs\\Lakshadweep\\metrics.json'

## Model Performance Dashboard

Create comprehensive performance comparison visualizations.

In [ ]:
# Create comprehensive performance dashboard
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. R² Score comparison
ax1 = axes[0, 0]
models = list(model_metrics.keys())
r2_scores = [model_metrics[model]['r2_score'] for model in models]
colors = plt.cm.viridis(np.linspace(0, 1, len(models)))

bars = ax1.bar(models, r2_scores, color=colors)
ax1.set_title('R² Score by Model', fontsize=14, fontweight='bold')
ax1.set_ylabel('R² Score')
ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, score in zip(bars, r2_scores):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. RMSE comparison
ax2 = axes[0, 1]
rmse_values = [model_metrics[model]['rmse'] for model in models]
bars = ax2.bar(models, rmse_values, color=colors)
ax2.set_title('RMSE by Model', fontsize=14, fontweight='bold')
ax2.set_ylabel('RMSE (meters)')
ax2.grid(True, alpha=0.3)

# Add value labels
for bar, rmse in zip(bars, rmse_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(rmse_values)*0.01, 
             f'{rmse:.2f}', ha='center', va='bottom', fontweight='bold')

# 3. MAE comparison  
ax3 = axes[1, 0]
mae_values = [model_metrics[model]['mae'] for model in models]
bars = ax3.bar(models, mae_values, color=colors)
ax3.set_title('Mean Absolute Error by Model', fontsize=14, fontweight='bold')
ax3.set_ylabel('MAE (meters)')
ax3.grid(True, alpha=0.3)

# Add value labels
for bar, mae in zip(bars, mae_values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(mae_values)*0.01, 
             f'{mae:.2f}', ha='center', va='bottom', fontweight='bold')

# 4. All metrics radar chart (simplified as table)
ax4 = axes[1, 1] 
ax4.axis('off')

# Create performance table
table_data = []
for model in models:
    metrics = model_metrics[model]
    table_data.append([
        model,
        f"{metrics['r2_score']:.3f}",
        f"{metrics['rmse']:.2f}",
        f"{metrics['mae']:.2f}",
        f"{metrics['mse']:.2f}"
    ])

table = ax4.table(cellText=table_data,
                  colLabels=['Model', 'R²', 'RMSE', 'MAE', 'MSE'],
                  cellLoc='center',
                  loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)

# Highlight best model row
best_model_idx = models.index(best_model_name) + 1  # +1 for header
for j in range(5):
    table[(best_model_idx, j)].set_facecolor('#90EE90')  # Light green

ax4.set_title('Performance Summary Table\n(Best model highlighted)', 
              fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(viz_dir / 'performance_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Created performance dashboard")

## Feature Importance Analysis

Analyze and visualize feature importance from the best performing model.

In [ ]:
# Load the best model for feature importance analysis
best_model_path = models_dir / f"{best_model_name}.joblib"
best_model_info_path = models_dir / f"{best_model_name}_info.json"

if best_model_path.exists() and best_model_info_path.exists():
    # Load model and info
    best_model = joblib.load(best_model_path)
    with open(best_model_info_path) as f:
        model_info = json.load(f)
    
    feature_names = model_info['feature_names']
    
    # Get feature importance (only for tree-based models)
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
        
        # Create feature importance visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # 1. Feature importance bar chart
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=True)
        
        colors = plt.cm.plasma(np.linspace(0, 1, len(importance_df)))
        bars = ax1.barh(importance_df['Feature'], importance_df['Importance'], color=colors)
        ax1.set_title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Importance Score')
        ax1.grid(True, alpha=0.3)
        
        # Add value labels
        for i, (bar, importance) in enumerate(zip(bars, importance_df['Importance'])):
            ax1.text(importance + max(importance_df['Importance'])*0.01, bar.get_y() + bar.get_height()/2,
                     f'{importance:.3f}', va='center', ha='left', fontweight='bold')
        
        # 2. Feature importance pie chart (top 6 features)
        top_features = importance_df.tail(6)
        colors_pie = plt.cm.Set3(np.linspace(0, 1, len(top_features)))
        
        wedges, texts, autotexts = ax2.pie(top_features['Importance'], 
                                           labels=top_features['Feature'],
                                           autopct='%1.1f%%',
                                           colors=colors_pie,
                                           startangle=90)
        ax2.set_title('Top 6 Most Important Features', fontsize=14, fontweight='bold')
        
        # Improve text readability
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'feature_importance.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Created feature importance analysis for {best_model_name}")
        print(f"📊 Most important feature: {importance_df.iloc[-1]['Feature']} ({importance_df.iloc[-1]['Importance']:.3f})")
        
    else:
        print(f"⚠️  Feature importance not available for {best_model_name} (not a tree-based model)")
        
else:
    print(f"⚠️  Best model files not found: {best_model_path}")

## Training Data Analysis

Analyze the synthetic depth data distribution and model training characteristics.

In [ ]:
# Load training data to analyze synthetic depth distribution
training_data_dir = project_dir / 'data' / 'sentinel' / region_slug / 'processed' / 'training_data'
feature_metadata_path = training_data_dir / 'feature_metadata.json'

if feature_metadata_path.exists():
    with open(feature_metadata_path) as f:
        training_metadata = json.load(f)
    
    # Create training data analysis visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Dataset size info
    ax1 = axes[0, 0]
    ax1.axis('off')
    
    info_text = f"""
    Dataset Summary
    
    Region: {training_metadata['region_name']}
    
    Samples: {training_metadata['n_samples']:,}
    Features: {training_metadata['n_features']}
    Water Pixels: {training_metadata['water_pixel_count']:,}
    
    Feature Names:
    {chr(10).join(['• ' + name for name in training_metadata['feature_names']])}
    """
    
    ax1.text(0.05, 0.95, info_text, transform=ax1.transAxes, fontsize=12,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
    
    # 2. Depth statistics
    depth_stats = training_metadata['depth_stats']
    ax2 = axes[0, 1]
    
    stats_labels = ['Min', 'Mean', 'Max', 'Std Dev']
    stats_values = [depth_stats['min'], depth_stats['mean'], 
                   depth_stats['max'], depth_stats['std']]
    
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']
    bars = ax2.bar(stats_labels, stats_values, color=colors)
    ax2.set_title('Synthetic Depth Statistics', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Depth (meters)')
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, stats_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(stats_values)*0.01,
                 f'{value:.1f}m', ha='center', va='bottom', fontweight='bold')
    
    # 3. Model count and types
    ax3 = axes[1, 0]
    model_names = list(model_metrics.keys())
    model_counts = [1] * len(model_names)  # Each model trained once
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))
    bars = ax3.bar(model_names, model_counts, color=colors)
    ax3.set_title('Models Trained', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Count')
    ax3.set_ylim(0, 2)
    
    # Add checkmarks
    for bar in bars:
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 '✓', ha='center', va='bottom', fontsize=20, color='green', fontweight='bold')
    
    # 4. Performance summary
    ax4 = axes[1, 1]
    
    performance_summary = f"""
    🏆 Best Model Performance
    
    Model: {best_model_name}
    R² Score: {best_r2:.4f}
    RMSE: {model_metrics[best_model_name]['rmse']:.2f} meters
    MAE: {model_metrics[best_model_name]['mae']:.2f} meters
    
    💡 Training Notes:
    • Synthetic depth data generated
    • Features scaled using StandardScaler  
    • 80/20 train-test split
    • Multiple ML algorithms tested
    """
    
    ax4.text(0.05, 0.95, performance_summary, transform=ax4.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen', alpha=0.8))
    ax4.axis('off')
    
    plt.tight_layout()
    plt.savefig(viz_dir / 'training_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Created training data analysis")
    
else:
    print("⚠️  Training metadata not found")

## Final Summary Report

Generate a comprehensive summary of the entire bathymetry analysis pipeline.

In [ ]:
# Generate final summary report
print("=" * 80)
print(f"🌊 SATELLITE-DERIVED BATHYMETRY ANALYSIS COMPLETE 🌊")
print("=" * 80)

print(f"\n📍 REGION: {region_name}")
print(f"📅 Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📊 DATASET SUMMARY:")
if 'training_metadata' in locals():
    print(f"   • Samples analyzed: {training_metadata['n_samples']:,}")
    print(f"   • Features extracted: {training_metadata['n_features']}")
    print(f"   • Water pixels: {training_metadata['water_pixel_count']:,}")
    print(f"   • Depth range: {training_metadata['depth_stats']['min']:.1f} - {training_metadata['depth_stats']['max']:.1f} meters")

print(f"\n🤖 MODELS TRAINED:")
for i, (model_name, metrics) in enumerate(model_metrics.items(), 1):
    status = "🏆" if model_name == best_model_name else "✅"
    print(f"   {i}. {status} {model_name}")
    print(f"      R² = {metrics['r2_score']:.4f}, RMSE = {metrics['rmse']:.2f}m, MAE = {metrics['mae']:.2f}m")

print(f"\n🎯 PIPELINE STAGES COMPLETED:")
pipeline_stages = [
    "01. Sentinel-2 Data Download ✅",
    "02. Data Preprocessing ✅", 
    "03. Feature Generation ✅",
    "04. Model Training ✅",
    "05. Performance Evaluation ✅",
    "06. Visualization Generation ✅"
]

for stage in pipeline_stages:
    print(f"   {stage}")

print(f"\n📁 OUTPUT FILES GENERATED:")
output_files = [
    f"Performance Dashboard: {viz_dir / 'performance_dashboard.png'}",
    f"Feature Importance: {viz_dir / 'feature_importance.png'}",
    f"Training Analysis: {viz_dir / 'training_analysis.png'}",
    f"Model Metrics: {output_dir / 'model_metrics.json'}",
    f"Performance Comparison: {output_dir / 'model_performance_comparison.csv'}"
]

for file_desc in output_files:
    print(f"   📄 {file_desc}")

print(f"\n💾 TRAINED MODELS SAVED:")
for model_name in model_metrics.keys():
    model_path = models_dir / f"{model_name}.joblib"
    if model_path.exists():
        print(f"   🤖 {model_name}: {model_path}")

print(f"\n🎉 SUCCESS! Bathymetry analysis pipeline completed successfully.")
print(f"🏆 Best performing model: {best_model_name} (R² = {best_r2:.4f})")
print("=" * 80)

# Save summary report
summary_report = {
    'analysis_date': pd.Timestamp.now().isoformat(),
    'region': region_name,
    'best_model': best_model_name,
    'best_r2': float(best_r2),
    'models_trained': list(model_metrics.keys()),
    'total_models': len(model_metrics),
    'output_directory': str(output_dir),
    'models_directory': str(models_dir),
    'visualizations_directory': str(viz_dir)
}

if 'training_metadata' in locals():
    summary_report.update({
        'dataset_size': training_metadata['n_samples'],
        'n_features': training_metadata['n_features'],
        'water_pixels': training_metadata['water_pixel_count'],
        'depth_range': training_metadata['depth_stats']
    })

summary_path = output_dir / 'pipeline_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary_report, f, indent=2)

print(f"\n📋 Summary report saved: {summary_path}")

## Advanced 3D/4D Visualizations Integration

Execute advanced visualization scripts from the visualisations directory to generate comprehensive 3D plots, heatmaps, and interactive analyses.

In [ ]:
# Advanced Visualizations Integration
print("🎨 Loading Advanced Visualization Extensions...")

try:
    # Import our advanced visualization system
    from src.visualize import run_advanced_visualizations
    
    # Create final showcase directory
    final_showcase_dir = output_dir / 'final_showcase'
    final_showcase_dir.mkdir(parents=True, exist_ok=True)
    
    # Run advanced visualizations
    print("🚀 Executing advanced visualization scripts...")
    advanced_files = run_advanced_visualizations(final_showcase_dir)
    
    if advanced_files:
        print(f"\n🎉 Advanced Visualization Files Generated:")
        for file_path in advanced_files:
            print(f"  ✅ {file_path}")
        
        # Update summary report with advanced visualizations
        if 'summary_report' in locals():
            summary_report['advanced_visualizations'] = advanced_files
            summary_report['total_visualizations'] = len(advanced_files)
            
            # Re-save updated summary
            with open(summary_path, 'w') as f:
                json.dump(summary_report, f, indent=2)
            
            print(f"\n📋 Updated summary report with {len(advanced_files)} advanced visualizations")
        
        # Show file sizes and types
        print(f"\n📊 Advanced Visualization Summary:")
        html_files = [f for f in advanced_files if f.endswith('.html')]
        png_files = [f for f in advanced_files if f.endswith('.png')]
        
        if html_files:
            print(f"  🌐 Interactive HTML plots: {len(html_files)}")
            for html_file in html_files[:3]:  # Show first 3
                print(f"    • {html_file}")
            if len(html_files) > 3:
                print(f"    • ... and {len(html_files) - 3} more")
                
        if png_files:
            print(f"  🖼️  Static PNG plots: {len(png_files)}")
            for png_file in png_files[:3]:  # Show first 3
                print(f"    • {png_file}")
            if len(png_files) > 3:
                print(f"    • ... and {len(png_files) - 3} more")
        
    else:
        print("ℹ️  No advanced visualization files were generated")
        print("    This is normal if:")
        print("    • No visualization scripts are present in visualisations/")
        print("    • Scripts encountered errors during execution")
        print("    • Data files are missing or in different locations")
        
except ImportError as e:
    print(f"⚠️  Could not load advanced visualization system: {e}")
    print("ℹ️  This is normal if the visualization module is not properly set up")
    print("    The basic visualizations above were still created successfully")
except Exception as e:
    print(f"❌ Error in advanced visualizations: {e}")
    print("⚠️  Advanced visualizations failed, but basic pipeline completed successfully")
    import traceback
    print("\nError details:")
    traceback.print_exc()

print("✅ Advanced visualization integration complete!")

## Final Output Directory Structure

Display the complete structure of generated files for easy access and reference.

In [ ]:
# Display final output directory structure
print("📁 GENERATED OUTPUT STRUCTURE:")
print("=" * 50)

def print_directory_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Print directory tree structure"""
    if current_depth > max_depth:
        return
        
    try:
        items = sorted(directory.iterdir())
        dirs = [item for item in items if item.is_dir()]
        files = [item for item in items if item.is_file()]
        
        # Print directories first
        for i, item in enumerate(dirs):
            is_last_dir = (i == len(dirs) - 1) and len(files) == 0
            print(f"{prefix}{'└── ' if is_last_dir else '├── '}{item.name}/")
            
            extension = "    " if is_last_dir else "│   "
            print_directory_tree(item, prefix + extension, max_depth, current_depth + 1)
        
        # Print files
        for i, item in enumerate(files):
            is_last = i == len(files) - 1
            size_mb = item.stat().st_size / (1024 * 1024)
            size_str = f" ({size_mb:.1f}MB)" if size_mb > 0.1 else f" ({item.stat().st_size}B)"
            
            print(f"{prefix}{'└── ' if is_last else '├── '}{item.name}{size_str}")
            
    except PermissionError:
        print(f"{prefix}[Permission Denied]")

# Start from output directory
print(f"{output_dir.name}/")
print_directory_tree(output_dir)

print("\n" + "=" * 50)
print("🎉 BATHYMETRY ANALYSIS PIPELINE COMPLETE!")
print(f"📊 Check {output_dir} for all results")
print(f"🌐 Interactive plots in {output_dir}/final_showcase/")
print(f"📈 Performance dashboards in {viz_dir}/")
print("=" * 50)

## Advanced 3D/4D Visualizations Integration

Execute advanced visualization scripts from the visualisations directory to generate comprehensive 3D plots, heatmaps, and interactive analyses.

In [ ]:
# Advanced Visualizations Integration
print("🎨 Loading Advanced Visualization Extensions...")

try:
    # Import our advanced visualization system
    from src.visualize import run_advanced_visualizations
    
    # Create final showcase directory
    final_showcase_dir = output_dir / 'final_showcase'
    final_showcase_dir.mkdir(parents=True, exist_ok=True)
    
    # Run advanced visualizations
    print(f"🚀 Executing advanced visualization scripts...")
    advanced_files = run_advanced_visualizations(final_showcase_dir)
    
    if advanced_files:
        print(f"\n🎉 Advanced Visualization Files Generated:")
        for file_path in advanced_files:
            print(f"  ✅ {file_path}")
        
        # Update summary report with advanced visualizations
        if 'summary_report' in locals():
            summary_report['advanced_visualizations'] = advanced_files
            summary_report['total_visualizations'] = len(advanced_files)
            
            # Re-save updated summary
            with open(summary_path, 'w') as f:
                json.dump(summary_report, f, indent=2)
            
            print(f"\n📋 Updated summary report with {len(advanced_files)} advanced visualizations")
        
        # Show file sizes and types
        print(f"\n📊 Advanced Visualization Summary:")
        html_files = [f for f in advanced_files if f.endswith('.html')]
        png_files = [f for f in advanced_files if f.endswith('.png')]
        
        if html_files:
            print(f"  🌐 Interactive HTML plots: {len(html_files)}")
            for html_file in html_files[:3]:  # Show first 3
                print(f"    • {html_file}")
            if len(html_files) > 3:
                print(f"    • ... and {len(html_files) - 3} more")
                
        if png_files:
            print(f"  🖼️  Static PNG plots: {len(png_files)}")
            for png_file in png_files[:3]:  # Show first 3
                print(f"    • {png_file}")
            if len(png_files) > 3:
                print(f"    • ... and {len(png_files) - 3} more")
        
    else:
        print("ℹ️  No advanced visualization files were generated")
        print("    This is normal if:")
        print("    • No visualization scripts are present in visualisations/")
        print("    • Scripts encountered errors during execution")
        print("    • Data files are missing or in different locations")
        
except ImportError as e:
    print(f"⚠️  Could not load advanced visualization system: {e}")
    print("ℹ️  This is normal if the visualization module is not properly set up")
    print("    The basic visualizations above were still created successfully")
except Exception as e:
    print(f"❌ Error in advanced visualizations: {e}")
    print("⚠️  Advanced visualizations failed, but basic pipeline completed successfully")
    import traceback
    print("\nError details:")
    traceback.print_exc()

print("✅ Advanced visualization integration complete!")

## Final Output Directory Structure

Display the complete structure of generated files for easy access and reference.

In [ ]:
# Display final output directory structure
print("📁 GENERATED OUTPUT STRUCTURE:")
print("=" * 50)

def print_directory_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Print directory tree structure"""
    if current_depth > max_depth:
        return
        
    try:
        items = sorted(directory.iterdir())
        dirs = [item for item in items if item.is_dir()]
        files = [item for item in items if item.is_file()]
        
        # Print directories first
        for i, item in enumerate(dirs):
            is_last_dir = (i == len(dirs) - 1) and len(files) == 0
            print(f"{prefix}{'└── ' if is_last_dir else '├── '}{item.name}/")
            
            extension = "    " if is_last_dir else "│   "
            print_directory_tree(item, prefix + extension, max_depth, current_depth + 1)
        
        # Print files
        for i, item in enumerate(files):
            is_last = i == len(files) - 1
            size_mb = item.stat().st_size / (1024 * 1024)
            size_str = f" ({size_mb:.1f}MB)" if size_mb > 0.1 else f" ({item.stat().st_size}B)"
            
            print(f"{prefix}{'└── ' if is_last else '├── '}{item.name}{size_str}")
            
    except PermissionError:
        print(f"{prefix}[Permission Denied]")

# Start from output directory
print(f"{output_dir.name}/")
print_directory_tree(output_dir)

print("\n" + "=" * 50)
print("🎉 BATHYMETRY ANALYSIS PIPELINE COMPLETE!")
print(f"📊 Check {output_dir} for all results")
print(f"🌐 Interactive plots in {output_dir}/final_showcase/")
print(f"📈 Performance dashboards in {viz_dir}/")
print("=" * 50)